In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb as db
import pandas as pd
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)

## Python and SQL - Workflow Analysis


In [ ]:
df_StpFcEntries = pd.read_excel('Tech_Datasets2024\StpFcEntries.xlsx')
df_contractPaths = pd.read_excel('Tech_Datasets2024\contractPaths.xlsx')
df_WorkflowTaskInfo = pd.read_excel('Tech_Datasets2024\WorkflowTaskInfo.xlsx')

In [ ]:
df_WorkflowTaskInfo.head()

In [ ]:
df_contractPaths.head()

In [ ]:
# print(df_StpFcEntries.columns)
df_StpFcEntries.head()

### Q2

#### Summarize the data by Family, Group and Type and output the number of contracts (not entries) that were processed by the contract workflow. 

Print the relevant columns:

In [ ]:
df1 = df_StpFcEntries[['status_taken', 'fc_id', 'fc_cfamily', 'fc_cgroup', 'fc_ctype']]
df1

Find the entries that have been processed:

In [ ]:
df1[df1['status_taken'].str.contains('Y')]

Finding Unique Values in a Pandas Dataframe
To find unique values in a Pandas dataframe, we can use the unique() function. This function returns an array of unique values in the dataframe. However, this function only returns unique values within a particular column or row. 

In [ ]:
unique_contracts = df1['fc_id'].unique()

print(unique_contracts)
print("Number of contracts processed by the contract workflow = ", len(unique_contracts))


### Q3

#### Output the following statistics per task:

Q3a) Wait-time Average

Q3a.1) First make a dataframe with the needed data:

In [ ]:
df2 = df_StpFcEntries[['status_taken', 'wait_time', 'xmlflow_status']]
df2 = df2[df2['status_taken'].str.contains('Y')]
df2 = df2[['xmlflow_status', 'wait_time']]
df2


Q3a.2) Combine/Add the wait times of entries under the same task.

In [ ]:
# Add the wait times of the entries under the same task

# To calculate the average we need to know the number of entries per task.
# To do this create an extra column of 1s. This is done so that when the row with 
# the same task are added together, we can see how many entries there were per task.

# Using DataFrame.insert() to add a column
df2.insert(2, "Ones", np.ones(df2.shape[0]), True)
print(df2.head())

df_waitTimePerTask = df2.groupby('xmlflow_status').sum()
df_waitTimePerTask

Q3a.3) Calculate the Average wait time *per task*

In [ ]:
df_AverageWaitTimePerTask = df_waitTimePerTask['wait_time']/df_waitTimePerTask['Ones']
df_AverageWaitTimePerTask = df_AverageWaitTimePerTask.reset_index()
df_AverageWaitTimePerTask = df_AverageWaitTimePerTask.rename(columns={0: 'Avg_Wait_Time'})
df_AverageWaitTimePerTask

plot histogram to view outliers

In [ ]:
sns.histplot(x = 'Avg_Wait_Time', data=df_AverageWaitTimePerTask, bins=50)
plt.show()

Q3b) Max Wait time *per task*

In [ ]:
df_MaxWaitTimePerTask = df2.groupby('xmlflow_status').max()

df_MaxWaitTimePerTask = df_MaxWaitTimePerTask['wait_time']
df_MaxWaitTimePerTask = df_MaxWaitTimePerTask.reset_index()
df_MaxWaitTimePerTask = df_MaxWaitTimePerTask.rename(columns={'wait_time': 'Max_wait_time'})

df_MaxWaitTimePerTask



Q3c) Average Processing Time *per task*

Q3c.1) First make a dataframe with the needed data:

In [ ]:
df_processingTimes = df_StpFcEntries[['status_taken', 'proc_time', 'xmlflow_status']]
df_processingTimes = df_processingTimes[df_processingTimes['status_taken'].str.contains('Y')]
df_processingTimes = df_processingTimes[['xmlflow_status', 'proc_time']]
df_processingTimes

Q3c.2) Combine/Add the Processing times of entries under the same task.

In [ ]:
# Add the processing times of the entries under the same task

# To calculate the average we need to know the number of entries per task.
# To do this create an extra column of 1s. This is done so that when rows with 
# the same task number are added together, the ones will be added together and we can see how many entries there were per task.

# Using DataFrame.insert() to add a column
df_processingTimes.insert(2, "Ones", np.ones(df_processingTimes.shape[0]), True)
print(df_processingTimes.head())

df_ProcTimePerTask = df_processingTimes.groupby('xmlflow_status').sum()
df_ProcTimePerTask = df_ProcTimePerTask.reset_index()
df_ProcTimePerTask

Q3c.3) Calculate the Average proc time *per task*

In [ ]:
df_AverageProcTimePerTask = df_ProcTimePerTask
df_AverageProcTimePerTask['avg_proc_time'] = df_AverageProcTimePerTask['proc_time']/df_AverageProcTimePerTask['Ones']
df_AverageProcTimePerTask

Q3d) Processing Time Max

In [ ]:
df_MaxProcTimePerTask = df_ProcTimePerTask[['xmlflow_status', 'proc_time']]
df_MaxProcTimePerTask = df_MaxProcTimePerTask.groupby('xmlflow_status').max()
df_MaxProcTimePerTask = df_MaxProcTimePerTask.rename(columns={'proc_time': 'max_proc_time'})
df_MaxProcTimePerTask = df_MaxProcTimePerTask.reset_index()

df_MaxProcTimePerTask

Q3e) Throughput (per second)

The Throughput(per second) per task was calculated by 1/the average processing time per task 

Frequency = 1/Period

Throughput = 1/Average proc time per task


In [ ]:
df_ThroughputPerTask = 1/(df_AverageProcTimePerTask/1000)
df_ThroughputPerTask

Q3f) Throughput (per minute)

In [ ]:
df_ThroughputPerTask_PerMinute = df_ThroughputPerTask*60
df_ThroughputPerTask_PerMinute


Q3g) Latency (per second)

Q3g.1) First make a dataframe with the needed data:

In [ ]:
df_Throughput = df_StpFcEntries[['status_taken', 'xmlflow_status', 'wait_time', 'proc_time']]
df_Throughput = df_Throughput[df_Throughput['status_taken'].str.contains('Y')]
df_Throughput = df_Throughput[['xmlflow_status', 'wait_time', 'proc_time']]
df_Throughput

Q3g.2) Combine/Add the Processing and Wait times of entries under the same task.

In [ ]:
df_ProcAndWaitTimePerTask = df_Throughput.groupby('xmlflow_status').sum()
df_ProcAndWaitTimePerTask

Q3g.3) Latency (per second)

In [ ]:
df_LatencyPerTask = df_ProcAndWaitTimePerTask['wait_time'] + df_ProcAndWaitTimePerTask['proc_time']
df_LatencyPerTask = df_LatencyPerTask.reset_index()
df_LatencyPerTask = df_LatencyPerTask.rename(columns = {0:'latency'})

df_LatencyPerTask['xmlflow_status'] = df_LatencyPerTask['xmlflow_status'].astype(str)
df_LatencyPerTask

##### Q4

Graphically show how many entries are waiting to be processed by each task in a bar chart. 



In [ ]:
df3 = df_StpFcEntries[['xmlflow_status', 'status_taken']]
df3 = df3[df3['status_taken'].str.contains('N')]

df3.insert(2, "Ones", np.ones(df3.shape[0]), True)

print(df3)


df3 = df3.groupby('xmlflow_status').sum()
df3 = df3.drop('status_taken', axis=1)
df3 = df3.reset_index()

df3

In [ ]:
df3['xmlflow_status'] = df3['xmlflow_status'].astype(str)


# Plot entries waiting to be processed
sns.barplot(x='Ones', y='xmlflow_status', data=df3, color="b")

##### Q5

Graphically display the top five tasks in terms of processing time in a line graph for the last week. 



In [ ]:
df4 = df_MaxProcTimePerTask
df4 = df4.sort_values('max_proc_time', ascending=False)

df4 = df4.head(5)

df4


In [ ]:
df4['xmlflow_status'] = df4['xmlflow_status'].astype(str)

sns.lineplot(y='max_proc_time', x='xmlflow_status', data=df4, color="b")

##### Q6

Write code to track the path of a contract in the workflow.



In [ ]:
df5 = df_StpFcEntries[['ts_time_long', 'status_taken', 'fc_id', 'xmlflow_status', 'fc_cfamily', 'fc_cgroup', 'fc_ctype']]
df5 = df5[df5['status_taken'].str.contains('Y')]
df5 = df5.sort_values('fc_id')
df5 

In [ ]:
df_fc_id = pd.read_excel('Tech_Datasets2024\contractPaths.xlsx')
df_fc_id = df_fc_id['fc_id']


for i in df_fc_id:
    print("")
    selected_rows = df5.loc[df5['fc_id'] == i]
    selected_rows = selected_rows.sort_values('ts_time_long')
    selected_rows
    family = selected_rows.iloc[0]['fc_cfamily']
    group = selected_rows.iloc[0]['fc_cgroup']
    print("Contract", i, "|", family, "|", group, ":", end="")
    for j in selected_rows['xmlflow_status']:
        print(j, end="->")

#### Q7) Data Analysis

Q7a.1) First make a dataframe with the needed data:

In [ ]:
df6 = df_StpFcEntries[['status_taken', 'proc_time', 'xmlflow_status', 'fc_ctype']]
df6 = df6[df6['status_taken'].str.contains('Y')]
df6 = df6[['fc_ctype', 'proc_time']]
df6

Q7a.2) Calculate the mean proc time per group:

In [ ]:
df6 = df6.groupby('fc_ctype').mean()
df6 = df6.reset_index()
df6


In [ ]:
df6 = df6.sort_values('proc_time', ascending=False)
df6

In [ ]:
df6 = df6.head(3)
df6['fc_ctype'] = df6['fc_ctype'].astype(str)
df6

sns.barplot(y='proc_time', x='fc_ctype', data=df6, color="b")

Q7b) Are there any tasks with bottlenecks?

Yes as there are many entries with status_taken field as 'N'. This means that we need to count the number of 'N's per task to see the number of entries backlogged per task.
Assumption: The term "bottlenecks" refers to the amount of entries backlogged per task.

In [ ]:
df7 = df_StpFcEntries[['xmlflow_status', 'status_taken']]
df7 = df7[df7['status_taken'].str.contains('N')]

# Using DataFrame.insert() to add a column
df7.insert(2, "Ones", np.ones(df7.shape[0]), True)

df7 = df7[['xmlflow_status', 'Ones']]

df7 = df7.groupby('xmlflow_status').sum()
df7 = df7.sort_values('Ones', ascending=False)

df7

In [ ]:
df7 = df7.reset_index()
df7['xmlflow_status'] = df7['xmlflow_status'].astype(str)
df7

sns.barplot(y='Ones', x='xmlflow_status', data=df7, color="b")

##### Q7c) Assume that the traders want to increase the volumes of FX Spot trades – the goals is to process 100 000 trades using a batch upload. 

Q7ci) Estimate how long it might take to process all the trades in the workflow? 

In [84]:
df8 = df_StpFcEntries[['proc_time', 'status_taken', 'fc_typology']]
df8 = df8[df8['status_taken'].str.contains('Y')]
df8 = df8[df8['fc_typology'].str.contains('Spot')]
df8

,proc_time,status_taken,fc_typology
937,4,Y,Spot
945,6,Y,Spot
950,5,Y,Spot
955,6,Y,Spot
960,4,Y,Spot
...,...,...,...
34479,1,Y,Spot
34480,1,Y,Spot
34481,5,Y,Spot
34487,5,Y,Spot


In [ ]:
df8 = df8[['proc_time']]
avgProcTime = df8.mean().values[0]
print("Average processing time = ", avgProcTime, "seconds")
print("Average processing time for 100000 trades = ", avgProcTime*1000, "seconds")

Average processing time =  24.404107320304735 seconds
Average processing time for 100000 trades =  24404.107320304734 seconds
